In [1]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL Link-Geo")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/04 16:25:48 WARN Utils: Your hostname, LAPTOP-VAB0S7AL resolves to a loopback address: 127.0.1.1; using 172.19.195.48 instead (on interface eth0)
22/09/04 16:25:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/09/04 16:25:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/04 16:25:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/04 16:25:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/04 16:25:50 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/09/04 16:25:50 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.



Removing duplicate postcode entries

Postcode linking file from : https://www.matthewproctor.com/australian_postcodes

In [2]:
aus_postcodes = spark.read.option('header', True).csv('../data/tables/postcode_data.csv')

In [3]:
# Drop duplicate postcode entries -> This should be discussed, as the current aggregation is just keeping the first..
aus_postcodes = aus_postcodes.dropDuplicates(['postcode'])
aus_postcodes.select('postcode').describe()
#aus_postcodes.describe()

summary,postcode
count,3167
mean,4036.5857278181243
stddev,1792.7031469578917
min,0200
max,9999


In [4]:
# Subset of useful features
aus_postcodes_subset = aus_postcodes.select(['postcode', 'SA2_MAINCODE_2016', 'SA3_CODE_2016', 'SA4_CODE_2016', 'LOCALITY', 'LAT', 'LONG', 'electoraterating'])
aus_postcodes_subset.limit(3)

postcode,SA2_MAINCODE_2016,SA3_CODE_2016,SA4_CODE_2016,LOCALITY,LAT,LONG,electoraterating
0851,702051067,70205,702,KATHERINE,-15.711424,137.050719,Rural
1159,117031337,11703,117,SYDNEY,-33.794883,151.268071,null
2069,121011402,12101,121,CASTLE COVE,-33.780246,151.195368,Inner Metropolitan


In [5]:
transactions = spark.read.parquet('../data/curated/rawdata.parquet')

In [6]:
# Join the datasets on postcode
transactions = transactions.join(aus_postcodes_subset, on="postcode", how="left")
#transactions = transactions.join(aus_postcodes_subset, on="postcode", how="inner")
transactions.limit(3)

22/09/04 16:26:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


postcode,merchant_abn,consumer_id,user_id,dollar_value,order_id,order_datetime,consumer_name,address,state,gender,merchant_name,tags,tag,revenue_level,take_rate,industry_category,agriculture,arts_and_recreation,info_media_and_telecommunications,rental_hiring_and_real_estate,retail_and_wholesale_trade,SA2_MAINCODE_2016,SA3_CODE_2016,SA4_CODE_2016,LOCALITY,LAT,LONG,electoraterating
2212,15912561882,36103,18983,235.75679877020391,3fdd2acd-3b77-4c2...,2021-06-18,Lawrence Tyler,3511 Victoria Meadow,NSW,Male,Nullam Nisl Insti...,"[[shoe shops], [b...",shoe shops,b,3.72,retail_and_wholes...,0,0,0,0,1,119011360,11901,119,REVESBY,-33.947286,151.014547,Inner Metropolitan
3053,15912561882,147322,9248,213.64355235308224,4b193807-a752-4f6...,2021-07-26,Alexander Skinner,35694 Kimberly Co...,VIC,Male,Nullam Nisl Insti...,"[[shoe shops], [b...",shoe shops,b,3.72,retail_and_wholes...,0,0,0,0,1,206041117,20604,206,CARLTON,-37.803569,144.966112,Inner Metropolitan
3354,15912561882,218510,23629,157.5666353970244,227da7f6-c6c5-445...,2021-04-27,Amanda Brown,4946 Veronica Lak...,VIC,Female,Nullam Nisl Insti...,"[[shoe shops], [b...",shoe shops,b,3.72,retail_and_wholes...,0,0,0,0,1,201011002,20101,201,BAKERY HILL,-37.560917,143.867158,null


Notes:
- Might not need to keep SA3,4; They are just a subset of SA2
- Currently duplicate postcodes are dropped -> the first entry is kept
- This might not be the best practice, but is done in order to get a distinct SA2 region as postcodes sometimes contain more than one region?
- => will most likely need aggregation for when looking at further external datasets e.g. ABS income data


In [7]:
#transactions.write.mode('overwrite').parquet('../data/curated/rawdata.parquet')
transactions.write.mode('overwrite').parquet('../data/curated/rawdata_w-geoinfo.parquet')